In [97]:
import numpy as np
import csv
import sys
import os

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [98]:
def plot_background_fct(send_data, recv_data, label, do_plot=True):
    # diffs in miliseconds
    diffs = (recv_data - send_data) * 1000
    if do_plot:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        ax1.plot(diffs, label="background FCT"+label) 
        ax1.set_title("Background FCT")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Miliseconds')
        ax1.set_xlabel('Request ID')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Queuing delay vs Time.png", format="png")
        fig.show()
    return np.mean(diffs), np.percentile(diffs,99)

In [99]:
def plot_foreground_fct(send_data, recv_data, label, do_plot=True):
    # diffs in micrseconds
    diffs = (recv_data - send_data) * 1000 * 1000
    if do_plot:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        ax1.plot(diffs, label="background FCT"+label) 
        ax1.set_title("Foreground FCT")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Microseconds')
        ax1.set_xlabel('Request ID')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Queuing delay vs Time.png", format="png")
        fig.show()
    return np.mean(diffs), np.percentile(diffs,99)

In [100]:
def plot_queue_size(data, label, do_plot=True):
    # http://www.mathcs.emory.edu/~cheung/Courses/558-old/Syllabus/90-NS/trace.html
    # Columns are: Time(s)-fromNode-toNode-SizeB-SizePack-Packsarrived(in interval)
    # -departed - dropped -3: same in bytes
    times = data[:,0]
    q_size_B, q_size_P = data[:,3], data[:,4]
    q_size_B = np.trim_zeros(q_size_B, trim='b')
    q_size_P = np.trim_zeros(q_size_P, trim='b')
    times = times[:q_size_B.shape[0]]
    
    q_size_B = np.trim_zeros(q_size_B, trim='f')
    q_size_P = np.trim_zeros(q_size_P, trim='f')
    
    times = times[times.shape[0] - q_size_B.shape[0]:]
    
    if do_plot:
        fig, (ax1,ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10,10))
        ax1.plot(times, q_size_B, label="q_size in bytes "+label) 
        ax1.set_title("Q size vs Time")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Bytes')
        ax1.set_xlabel('Seconds')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        ax2.plot(times, q_size_P, label="q_size in packets "+label) 
        ax2.set_title("Q size vs Time")
        ax2.legend(loc='upper left')
        ax2.set_ylabel('Packets')
        ax2.set_xlabel('Seconds')
        ax2.grid(which='major', linestyle="-")
        ax2.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Q_size vs Time.png", format="png")
        fig.show()
    #avg_q_occup = np.mean(q_size_B)
    #print("Mon_mean: " + str(avg_q_occup))
    q_size_B = q_size_B/1000.0
    return np.mean(q_size_B), np.mean(q_size_P), np.percentile(q_size_B,99), np.percentile(q_size_P, 99)

In [101]:
def plot_queuing_time(data, label, link_speed, do_plot=True):
    """
    Link speed in Gbps
    results in microseconds
    """
    times = data[:,0]
    q_size_B, q_size_P = data[:,3], data[:,4]
    q_size_B = np.trim_zeros(q_size_B, trim='b')
    times = times[:q_size_B.shape[0]]
    q_size_B = np.trim_zeros(q_size_B, trim='f')
    times = times[times.shape[0] - q_size_B.shape[0]:]
    
    link_speed = link_speed * 1000.0 * 1000.0 * 1000.0
    q_time = q_size_B/(link_speed/8.0) * 1000.0 * 1000.0
    
    if do_plot:
        fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
        ax1.plot(times, q_time, label="queuing delay "+label) 
        ax1.set_title("Queuing delay vs Time")
        ax1.legend(loc='upper left')
        ax1.set_ylabel('Queuing microseconds')
        ax1.set_xlabel('Simulation Seconds')
        ax1.grid(which='major', linestyle="-")
        ax1.grid(which='minor', linestyle='--')

        fig.tight_layout()   

        fig.savefig("Queuing delay vs Time.png", format="png")
        fig.show()
    return np.mean(q_time), np.percentile(q_time, 99)

In [102]:
def print_results():
    print("Mean Q length in Kbytes {0} and in packets {1}\n99th percentile in Kbytes {2} and in packets {3}\n".format(bytes_mean, pkts_mean, bytes_99, pkts_99))
    print("Mean delay in us {0} and 99th percentile delay in us {1}\n".format(delay_mean, delay_99))
    print("Background FCT (ms): mean: {0}, 99th %: {1}\n".format(bkg_mean, bkg_99))
    print("Foreground FCT (us): mean: {0}, 99th %: {1}\n".format(frg_mean, frg_99))
    
    #print("Mean thrpt in Mbps {0} and 99th percentile thrpt in Mbps {1}".format(thrpt_mean, thrpt_99))    

# Paper reproduction
## 20% bottlenecked link load 

In [103]:
results_folders = ["/TCP/", "/DCTCP30/", "/DCTCP6_pacer/", "/DCTCP_pacer_PQ/"]
latency = dict()
bkg_fct = dict()
frg_fct = dict()

for folder in results_folders:
    latency[folder] = []
    bkg_fct[folder] = []
    frg_fct[folder] = []

In [104]:
simul_folder = "nofanout_20_9flows"

for folder in results_folders:
    path = simul_folder + folder
    q_mon = np.genfromtxt(path+"q_mon", delimiter=" ")
    bytes_mean, pkts_mean, bytes_99, pkts_99 = plot_queue_size(q_mon, folder, do_plot=False)
    delay_mean, delay_99 = plot_queuing_time(q_mon, folder, 1, do_plot=False)
    latency[folder].append((delay_mean, delay_99))
    
    bkg_send = np.genfromtxt(path+"send_times|background.csv", delimiter=" ")
    bkg_recv = np.genfromtxt(path+"rec_times|background.csv", delimiter=" ")
    bkg_mean, bkg_99 = plot_background_fct(bkg_send, bkg_recv, folder, do_plot=False)
    bkg_fct[folder].append((bkg_mean,bkg_99))
    
    frg_send = np.genfromtxt(path+"send_times|foreground.csv", delimiter=" ")
    frg_recv = np.genfromtxt(path+"rec_times|foreground.csv", delimiter=" ")
    frg_mean, frg_99 = plot_foreground_fct(frg_send, frg_recv, folder, do_plot=False)
    frg_fct[folder].append((frg_mean,frg_99))
    print(folder)
    print_results()

/TCP/
Mean Q length in Kbytes 10.774666688855316 and in packets 10.885744478247267
99th percentile in Kbytes 285.5532155044199 and in packets 288.54015191138933

Mean delay in us 86.19733351084253 and 99th percentile delay in us 2284.425724035359

Background FCT (ms): mean: 101.79763414446701, 99th %: 258.93551357355244

Foreground FCT (us): mean: 141.7319050942629, 99th %: 2387.537782686878

/DCTCP30/
Mean Q length in Kbytes 1.582409142971607 and in packets 1.598731741864602
99th percentile in Kbytes 30.232331470152005 and in packets 30.54226119317228

Mean delay in us 12.659273143772856 and 99th percentile delay in us 241.85865176121604

Background FCT (ms): mean: 102.94470360941773, 99th %: 267.2092007572399

Foreground FCT (us): mean: 65.21593668968896, 99th %: 828.6993371079634

/DCTCP6_pacer/
Mean Q length in Kbytes 0.7036849228360383 and in packets 0.7109561763990122
99th percentile in Kbytes 21.268981103215935 and in packets 21.485982702526847

Mean delay in us 5.62947938268830

## 40% bottlenecked link load 

In [105]:
simul_folder = "nofanout_40_9flows"

for folder in results_folders:
    path = simul_folder + folder
    q_mon = np.genfromtxt(path+"q_mon", delimiter=" ")
    bytes_mean,pkts_mean,bytes_99,pkts_99 = plot_queue_size(q_mon, folder, do_plot=False)
    delay_mean, delay_99 = plot_queuing_time(q_mon, folder, 1, do_plot=False)
    latency[folder].append((delay_mean, delay_99))
    
    
    bkg_send = np.genfromtxt(path+"send_times|background.csv", delimiter=" ")
    bkg_recv = np.genfromtxt(path+"rec_times|background.csv", delimiter=" ")
    bkg_mean, bkg_99 = plot_background_fct(bkg_send, bkg_recv, folder, do_plot=False)
    bkg_fct[folder].append((bkg_mean,bkg_99))
    
    
    frg_send = np.genfromtxt(path+"send_times|foreground.csv", delimiter=" ")
    frg_recv = np.genfromtxt(path+"rec_times|foreground.csv", delimiter=" ")
    frg_mean, frg_99 = plot_foreground_fct(frg_send, frg_recv, folder, do_plot=False)
    frg_fct[folder].append((frg_mean,frg_99))
    
    
    
    print(folder)
    print_results()

/TCP/
Mean Q length in Kbytes 44.0350871854736 and in packets 44.49438107921128
99th percentile in Kbytes 426.53155550045904 and in packets 430.86176231899356

Mean delay in us 352.2806974837888 and 99th percentile delay in us 3412.252444003673

Background FCT (ms): mean: 134.22350802345144, 99th %: 447.762181889981

Foreground FCT (us): mean: 430.20916449196415, 99th %: 3560.0888624011154

/DCTCP30/
Mean Q length in Kbytes 5.997761032481946 and in packets 6.060362497412727
99th percentile in Kbytes 43.11384228192006 and in packets 43.55231458383544

Mean delay in us 47.98208825985556 and 99th percentile delay in us 344.91073825536046

Background FCT (ms): mean: 138.9012526921508, 99th %: 423.3926133154763

Foreground FCT (us): mean: 109.67474404010161, 99th %: 965.3521214642069

/DCTCP6_pacer/
Mean Q length in Kbytes 2.531843348840745 and in packets 2.55833630106587
99th percentile in Kbytes 28.16265754336765 and in packets 28.458508454660517

Mean delay in us 20.25474679072596 and 99

## 60% bottlenecked link load

In [106]:
simul_folder = "nofanout_60_9flows"

for folder in results_folders:
    path = simul_folder + folder
    q_mon = np.genfromtxt(path+"q_mon", delimiter=" ")
    bytes_mean,pkts_mean,bytes_99,pkts_99 = plot_queue_size(q_mon, folder, do_plot=False)
    delay_mean, delay_99 = plot_queuing_time(q_mon, folder, 1, do_plot=False)
    latency[folder].append((delay_mean, delay_99))
    
    
    bkg_send = np.genfromtxt(path+"send_times|background.csv", delimiter=" ")
    bkg_recv = np.genfromtxt(path+"rec_times|background.csv", delimiter=" ")
    bkg_mean, bkg_99 = plot_background_fct(bkg_send, bkg_recv, folder, do_plot=False)
    bkg_fct[folder].append((bkg_mean,bkg_99))
    
    
    frg_send = np.genfromtxt(path+"send_times|foreground.csv", delimiter=" ")
    frg_recv = np.genfromtxt(path+"rec_times|foreground.csv", delimiter=" ")
    frg_mean, frg_99 = plot_foreground_fct(frg_send, frg_recv, folder, do_plot=False)
    frg_fct[folder].append((frg_mean,frg_99))
    
    
    print(folder)
    print_results()

/TCP/
Mean Q length in Kbytes 101.46689781627384 and in packets 102.5355332982317
99th percentile in Kbytes 464.75284139133 and in packets 469.4904869718644

Mean delay in us 811.7351825301907 and 99th percentile delay in us 3718.02273113064

Background FCT (ms): mean: 206.2282459162474, 99th %: 824.2336581929637

Foreground FCT (us): mean: 869.3429159026326, 99th %: 3791.5498266876584

/DCTCP30/
Mean Q length in Kbytes 12.804596881417107 and in packets 12.93949101421719
99th percentile in Kbytes 50.59961207240616 and in packets 51.15488183646201

Mean delay in us 102.43677505133685 and 99th percentile delay in us 404.7968965792493

Background FCT (ms): mean: 218.29191099266106, 99th %: 723.9846351209328

Foreground FCT (us): mean: 160.69834434703375, 99th %: 897.9092474859876

/DCTCP6_pacer/
Mean Q length in Kbytes 4.893057148611924 and in packets 4.9446610390217955
99th percentile in Kbytes 31.966649824767785 and in packets 32.31266139459839

Mean delay in us 39.144457188895394 and 9

In [107]:
print(latency)
print("------------------")
print(bkg_fct)
print("------------------")
print(frg_fct)

{'/TCP/': [(86.19733351084253, 2284.425724035359), (352.2806974837888, 3412.252444003673), (811.7351825301907, 3718.02273113064)], '/DCTCP30/': [(12.659273143772856, 241.85865176121604), (47.98208825985556, 344.91073825536046), (102.43677505133685, 404.7968965792493)], '/DCTCP6_pacer/': [(5.629479382688307, 170.15184882572748), (20.25474679072596, 225.3012603469412), (39.144457188895394, 255.73319859814228)], '/DCTCP_pacer_PQ/': [(0.3482664532322219, 9.744798565233731), (2.8381942744212583, 57.836609872545765), (12.856125942355781, 106.0808379412392)]}
------------------
{'/TCP/': [(101.79763414446701, 258.93551357355244), (134.22350802345144, 447.762181889981), (206.2282459162474, 824.2336581929637)], '/DCTCP30/': [(102.94470360941773, 267.2092007572399), (138.9012526921508, 423.3926133154763), (218.29191099266106, 723.9846351209328)], '/DCTCP6_pacer/': [(104.54532787048063, 276.8941435395714), (143.06387169839968, 430.7729404560873), (225.98879908095554, 735.9145466559312)], '/DCTCP_